In [1]:
from dataclasses import dataclass, field

@dataclass
class CallContext:
    customer_name: str | None = None
    interested: bool = False
    busy: bool = False
    rejected: bool = False
    preferred_location: str | None = None
    property_type: str | None = None
    budget: str | None = None
    follow_up_time: str | None = None
    raw_notes: list[str] = field(default_factory=list)

    @property
    def outcome_status(self) -> str:
        if self.interested:
            return "INTERESTED"
        if self.busy:
            return "BUSY - CALL LATER"
        if self.rejected:
            return "NOT INTERESTED"
        return "UNKNOWN"

    @property
    def summary_notes(self) -> str:
        notes = []

        if self.customer_name:
            notes.append(f"Customer name: {self.customer_name}")

        if self.preferred_location:
            notes.append(f"Preferred location: {self.preferred_location}")

        if self.property_type:
            notes.append(f"Property type: {self.property_type}")

        if self.budget:
            notes.append(f"Budget: {self.budget}")

        if self.raw_notes:
            notes.append("Extra notes: " + " | ".join(self.raw_notes))

        if not notes:
            return "No additional details collected."
        return " | ".join(notes)


In [2]:
def detect_intent(user_text: str) -> str:
    text = user_text.lower()

    interested_keywords = ["yes", "interested", "okay", "sure", "go ahead", "tell me"]
    busy_keywords = ["busy", "later", "not now", "call back", "another time"]
    reject_keywords = ["no", "not interested", "stop", "don't call", "never"]

    if any(word in text for word in interested_keywords):
        return "INTERESTED"
    if any(word in text for word in busy_keywords):
        return "BUSY"
    if any(word in text for word in reject_keywords):
        return "REJECT"

    return "UNKNOWN"


In [3]:
def agent_greeting():
    print("Agent: Hello! This is an automated assistant calling from Hookfish Realty.")
    print("Agent: You had shown interest in real-estate properties earlier.")
    print("Agent: Is this a good time to talk about a property that might fit your needs?")


def handle_first_response(context: CallContext, user_text: str) -> str:
    intent = detect_intent(user_text)

    if intent == "INTERESTED":
        context.interested = True
        print("\nAgent: That’s great to hear! Let me ask some quick questions.")
        return "INTERESTED_FLOW"

    elif intent == "BUSY":
        context.busy = True
        print("\nAgent: No worries, I understand you're busy.")
        print("Agent: When would be a good time for us to call back?")
        return "BUSY_FLOW"

    elif intent == "REJECT":
        context.rejected = True
        print("\nAgent: Understood. We'll avoid disturbing you.")
        return "REJECT_FLOW"

    else:
        print("\nAgent: I didn’t understand that.")
        print("Agent: Are you interested, busy, or not interested?")
        return "CLARIFY"


In [4]:
def run_interested_flow(context: CallContext):
    print("\nAgent: What is your preferred location?")
    location = input("Customer: ").strip()
    if location:
        context.preferred_location = location

    print("\nAgent: What type of property are you looking for?")
    print("Agent: (For example: apartment, villa, independent house, or plot)")
    property_type = input("Customer: ").strip()
    if property_type:
        context.property_type = property_type

    print("\nAgent: And what is your budget range?")
    budget = input("Customer: ").strip()
    if budget:
        context.budget = budget


    print("\nAgent: Thank you! Our team will contact you soon with suitable properties.")
    print("Agent: Have a nice day!")


def run_busy_flow(context: CallContext):
    follow_up = input("Customer (follow-up time): ").strip()
    if follow_up:
        context.follow_up_time = follow_up

    print("\nAgent: Great, I have noted that.")
    print("Agent: We'll contact you at that time.")
    print("Agent: Have a good day!")


def run_reject_flow(context: CallContext):
    print("\nAgent: Thank you for your time.")
    print("Agent: Feel free to reach out if you change your mind.")
    print("Agent: Goodbye!")


def clarify_and_route(context: CallContext):
    user_text = input("Customer: ")
    next_state = handle_first_response(context, user_text)

    if next_state == "INTERESTED_FLOW":
        run_interested_flow(context)
    elif next_state == "BUSY_FLOW":
        run_busy_flow(context)
    elif next_state == "REJECT_FLOW":
        run_reject_flow(context)
    else:
        print("\nAgent: I'm still confused. Ending call.")
        print("Agent: Have a good day!")


In [5]:
def run_call_simulation():
    context = CallContext()

    print(" Hookfish AI Calling Agent \n")

    name = input("Enter customer name : ").strip()
    if name:
        context.customer_name = name

    print("\n CALL STARTED \n")
    agent_greeting()

    first_reply = input("Customer: ")
    next_state = handle_first_response(context, first_reply)

    if next_state == "INTERESTED_FLOW":
        run_interested_flow(context)
    elif next_state == "BUSY_FLOW":
        run_busy_flow(context)
    elif next_state == "REJECT_FLOW":
        run_reject_flow(context)
    elif next_state == "CLARIFY":
        clarify_and_route(context)

    print("\n CALL SUMMARY ")
    print("Outcome:", context.outcome_status)
    print("Follow-up time:", context.follow_up_time or "None")
    print("Notes:", context.summary_notes)
    print("\n--- CALL ENDED ---")


In [7]:
run_call_simulation()


=== Hookfish AI Calling Agent - Simulation ===

Enter mock customer name (optional): Anu

--- CALL STARTED ---

Agent: Hello! This is an automated assistant calling from Hookfish Realty.
Agent: You had shown interest in real-estate properties earlier.
Agent: Is this a good time to talk about a property that might fit your needs?
Customer: Maybe

Agent: I didn’t understand that.
Agent: Are you interested, busy, or not interested?
Customer: Busy

Agent: No worries, I understand you're busy.
Agent: When would be a good time for us to call back?
Customer (follow-up time): after 2 hours

Agent: Great, I have noted that.
Agent: We'll contact you at that time.
Agent: Have a good day!

--- CALL SUMMARY ---
Outcome: BUSY - CALL LATER
Follow-up time: after 2 hours
Notes: Customer name: Anu

--- CALL ENDED ---
